# Functional annotations for variants

This notebook show how to use the genome annotations and gene models to translate the variant genomic coordinates into functional annotations.


- runtime: 30m
- recommended instance: mem1_ssd1_v2_x16
- estimated cost: <£0.70

This notebook depends on:
* **Bioconductor install**
* **Notebook 203** - height_signif_snp.csv

## Install required R packages

Function `p_load` from `pacman` loads packages into R.
If a given package missing it will be automatically installed - this can take a considerable amount of time for packages that need C or FORTRAN code compilation.

The following packages are needed to run this notebook:

- `dplyr` - tabular data manipulation in R, require to pre-process and filtering of phenotypic data
- `readr` - read and write tabular file formats: CSV, TsSV, TDF, etc.
- `skimr` - provide summary statistics about variables in data frames, `tibble` structures, data tables and vectors
- `gprofiler2` - A tool set for functional enrichment analysis and visualization of genes and variants
- `VariantAnnotation` - Bioconductor package for variant annotations 
- `TxDb.Hsapiens.UCSC.hg38.knownGene` - gene position for hg38 human genome release
- `BSgenome.Hsapiens.UCSC.hg38` - the DNA sequence of hg38 human genome release



In [ ]:
if(!require(pacman)) install.packages("pacman")
pacman::p_load(dplyr, skimr, readr, gprofiler2)

In [ ]:
# Install BioConductor contingent on R version
if(as.double(R.version$minor) < 3.0) {version <- '3.16'} else {version <- '3.17'}

if(!require(GenomicRanges)) BiocManager::install("GenomicRanges", version=version, ask=FALSE)
if(!require(VariantAnnotation)) BiocManager::install("VariantAnnotation", version=version, ask=FALSE)
if(!require(TxDb.Hsapiens.UCSC.hg38.knownGene)) BiocManager::install("TxDb.Hsapiens.UCSC.hg38.knownGene", version=version, ask=FALSE)
if(!require(BSgenome.Hsapiens.UCSC.hg38)) BiocManager::install("BSgenome.Hsapiens.UCSC.hg38", version=version, ask=FALSE)

In [3]:
pacman::p_load(TxDb.Hsapiens.UCSC.hg38.knownGene, BSgenome.Hsapiens.UCSC.hg38, VariantAnnotation, GenomicRanges)

## Load output of GWAS

In the first step, we get a list of variants. In the following example, we use the list of significant variants from GWAS on participant height example from **Notebook 203**.

In [4]:
system('dx download gwas/height_signif_snp.csv')
snp <- readr::read_csv('./height_signif_snp.csv', show_col_types = FALSE)
head(snp)

chromosome,marker.ID,genetic.dist,physical.pos,allele1,allele2,estim,std.err,score,p.value,fdr
<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6,6:52438563:G:A,0,52438563,A,G,-2.824782,0.7257148,-3.892413,0.0001775838,0.6689582
7,7:105508146:G:A,0,105508146,A,G,5.109052,1.4288079,3.575744,0.0005706030,0.7319412
1,1:39753608:T:C,0,39753608,C,T,1.224790,0.3431045,3.569729,0.0005829105,0.7319412


## Construct the GenomicRanges object from the list of variants

In this step, we use the genomic coordinates (chromosome and physical position) of variants to construct the GenomicRanges object.

In [5]:
snp_gr <- makeGRangesFromDataFrame(
    snp, 
    seqnames.field = 'chromosome', 
    start.field = 'physical.pos', 
    end.field = 'physical.pos', 
    keep.extra.columns = TRUE)

The GRanges object consists of 3 mandatory fields: `seqnames` - the name of the chromosome, `ranges` - position on the chromosome and `strand` - the strand, where `*` denote any strand.
In addition, there can be an arbitrary number of additional annotation fields. We use them to store marker ID, the information about alleles and statistics from GWAS analysis. 

In [6]:
head(snp_gr)

GRanges object with 3 ranges and 9 metadata columns:
      seqnames    ranges strand |       marker.ID genetic.dist     allele1
         <Rle> <IRanges>  <Rle> |     <character>    <numeric> <character>
  [1]        6  52438563      * |  6:52438563:G:A            0           A
  [2]        7 105508146      * | 7:105508146:G:A            0           A
  [3]        1  39753608      * |  1:39753608:T:C            0           C
          allele2     estim   std.err     score     p.value       fdr
      <character> <numeric> <numeric> <numeric>   <numeric> <numeric>
  [1]           G  -2.82478  0.725715  -3.89241 0.000177584  0.668958
  [2]           G   5.10905  1.428808   3.57574 0.000570603  0.731941
  [3]           T   1.22479  0.343105   3.56973 0.000582910  0.731941
  -------
  seqinfo: 3 sequences from an unspecified genome; no seqlengths

Next, we convert a `GenomicRanges` structure to `VariantRanges` class.  `VRanges` structure is a specialized extension of `GRanges`, designed specifically to hold information about genomic variation. 

In [7]:
vr <- VRanges(
    seqnames = seqnames(snp_gr),
    ranges = ranges(snp_gr),
    ref = snp_gr$allele1, 
    alt = snp_gr$allele2)

seqlevelsStyle(vr) <- seqlevelsStyle(TxDb.Hsapiens.UCSC.hg38.knownGene)

In [8]:
head(vr)

VRanges object with 3 ranges and 0 metadata columns:
      seqnames    ranges strand         ref              alt     totalDepth
         <Rle> <IRanges>  <Rle> <character> <characterOrRle> <integerOrRle>
  [1]     chr6  52438563      *           A                G           <NA>
  [2]     chr7 105508146      *           A                G           <NA>
  [3]     chr1  39753608      *           C                T           <NA>
            refDepth       altDepth   sampleNames softFilterMatrix
      <integerOrRle> <integerOrRle> <factorOrRle>         <matrix>
  [1]           <NA>           <NA>          <NA>                 
  [2]           <NA>           <NA>          <NA>                 
  [3]           <NA>           <NA>          <NA>                 
  -------
  seqinfo: 3 sequences from an unspecified genome; no seqlengths
  hardFilters: NULL

## Predict coding variants

This function returns the amino acid coding for variants that fall completely `within' a coding region For further information on predictCoding click [here](https://www.rdocumentation.org/packages/VariantAnnotation/versions/1.18.5/topics/predictCoding)

In [11]:
coding <- predictCoding(vr, TxDb.Hsapiens.UCSC.hg38.knownGene, BSgenome.Hsapiens.UCSC.hg38)
head(coding)

GRanges object with 6 ranges and 12 metadata columns:
      seqnames    ranges strand |      varAllele    CDSLOC    PROTEINLOC
         <Rle> <IRanges>  <Rle> | <DNAStringSet> <IRanges> <IntegerList>
  [1]     chr6  52438563      + |              G       221            74
  [2]     chr6  52438563      + |              G       221            74
  [3]     chr6  52438563      + |              G       488           163
  [4]     chr6  52438563      + |              G       545           182
  [5]     chr6  52438563      + |              G       545           182
  [6]     chr6  52438563      + |              G       545           182
        QUERYID        TXID                 CDSID      GENEID CONSEQUENCE
      <integer> <character>         <IntegerList> <character>    <factor>
  [1]         1       82079 89667,89665,89666,...      114327  synonymous
  [2]         1       82080 89667,89665,89666,...      114327  synonymous
  [3]         1       82087 89667,89665,89666,...      114327  syn

## Locate variants

We can assess the variant location with respect to gene function [more info](https://www.rdocumentation.org/packages/VariantAnnotation/versions/1.18.5/topics/locateVariants). 
In the examples below we select a different classes of variants based on functional annotation.

###  Coding variants

In [12]:
cds <- locateVariants(vr, TxDb.Hsapiens.UCSC.hg38.knownGene, CodingVariants())
head(cds)

GRanges object with 6 ranges and 9 metadata columns:
      seqnames    ranges strand | LOCATION  LOCSTART    LOCEND   QUERYID
         <Rle> <IRanges>  <Rle> | <factor> <integer> <integer> <integer>
  [1]     chr6  52438563      + |   coding       221       221         1
  [2]     chr6  52438563      + |   coding       221       221         1
  [3]     chr6  52438563      + |   coding       488       488         1
  [4]     chr6  52438563      + |   coding       545       545         1
  [5]     chr6  52438563      + |   coding       545       545         1
  [6]     chr6  52438563      + |   coding       545       545         1
             TXID                 CDSID      GENEID       PRECEDEID
      <character>         <IntegerList> <character> <CharacterList>
  [1]       82079 89667,89665,89666,...      114327                
  [2]       82080 89667,89665,89666,...      114327                
  [3]       82087 89667,89665,89666,...      114327                
  [4]       82092 89667

### Five UTR variants

In [13]:
five <- locateVariants(vr, TxDb.Hsapiens.UCSC.hg38.knownGene, FiveUTRVariants())
head(five)

'select()' returned 1:1 mapping between keys and columns



GRanges object with 1 range and 9 metadata columns:
      seqnames    ranges strand | LOCATION  LOCSTART    LOCEND   QUERYID
         <Rle> <IRanges>  <Rle> | <factor> <integer> <integer> <integer>
  [1]     chr6  52438563      + |  fiveUTR       429       429         1
             TXID         CDSID      GENEID       PRECEDEID        FOLLOWID
      <character> <IntegerList> <character> <CharacterList> <CharacterList>
  [1]       82109                    114327                                
  -------
  seqinfo: 3 sequences from an unspecified genome; no seqlengths

### Variants overlapping splice sites

In [14]:
splice <- locateVariants(vr, TxDb.Hsapiens.UCSC.hg38.knownGene, SpliceSiteVariants())
head(splice)

GRanges object with 0 ranges and 9 metadata columns:
   seqnames    ranges strand | LOCATION  LOCSTART    LOCEND   QUERYID      TXID
      <Rle> <IRanges>  <Rle> | <factor> <integer> <integer> <integer> <integer>
           CDSID      GENEID       PRECEDEID        FOLLOWID
   <IntegerList> <character> <CharacterList> <CharacterList>
  -------
  seqinfo: no sequences

### Intronic variants

In [15]:
intron <- locateVariants(vr, TxDb.Hsapiens.UCSC.hg38.knownGene, IntronVariants())
head(intron)

'select()' returned 1:1 mapping between keys and columns



GRanges object with 1 range and 9 metadata columns:
      seqnames    ranges strand | LOCATION  LOCSTART    LOCEND   QUERYID
         <Rle> <IRanges>  <Rle> | <factor> <integer> <integer> <integer>
  [1]     chr6  52438563      - |   intron     17868     17868         1
             TXID         CDSID      GENEID       PRECEDEID        FOLLOWID
      <character> <IntegerList> <character> <CharacterList> <CharacterList>
  [1]       73855                    729506                                
  -------
  seqinfo: 3 sequences from an unspecified genome; no seqlengths

head(intron)

## Summaries functional annotations

We can summaries the number of the variants in functional classes

In [16]:
lengths(list(cds=cds, five=five, splice=splice, intron=intron))

cds   five splice intron 
    20      1      0      1

## Get more info about coding genes

This function will convert coding Gene IDs to Ensembl IDs, gene names and short functional descriptions.

In [17]:
gconvert(query = unique(coding$GENEID), organism = "hsapiens", numeric_ns = 'ENTREZGENE_ACC')

input_number,input,target_number,target,name,description,namespace
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,114327,1.1,ENSG00000096093,EFHC1,EF-hand domain containing 1 [Source:HGNC Symbol;Acc:HGNC:16406],ENTREZGENE_ACC
2,54517,2.1,ENSG00000091127,PUS7,pseudouridine synthase 7 [Source:HGNC Symbol;Acc:HGNC:26033],ENTREZGENE_ACC
